In [1]:
%matplotlib inline
import sys
sys.path.append("/home/bianyuan/workspace/Grab_TNGgalaxy/")
# sys.path.append("./Grab_TNGgalaxy")
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
%matplotlib widget
run = 'TNG50'
basePath = '/media/bianyuan/data-TNG-1/' + run + '-1/output'
from astroML.plotting import setup_text_plots
#Lets text in plots use latex
# setup_text_plots(usetex=True)

/home/bianyuan/.conda/envs/py310/lib/python3.10/site-packages/astroML/linear_model/linear_regression_errors.py:10: UserWarning: LinearRegressionwithErrors requires PyMC3 to be installed
  warnings.warn('LinearRegressionwithErrors requires PyMC3 to be installed')


In [2]:
'''Tranfer SnapNum to Redshift'''

import numpy as np
import h5py, scipy
from pynbody import units as units_conv

Redshift_snapshot = np.loadtxt('/home/bianyuan/workspace/data/Redshift_snapshot.txt',comments='Snapshot')

def redshift_snapshot(snapshots, datafile='/home/bianyuan/workspace/data/Redshift_snapshot.txt', return_opt=None):
    snapshots0, redshifts0 = np.loadtxt(datafile, skiprows=1, unpack=True, usecols=[0, 2])
    n_snap = np.size(snapshots)
    if n_snap == 1: 
        redshifts = redshifts0[np.where(snapshots0 == snapshots)][0]
    elif n_snap >1:
        redshifts = np.zeros(n_snap)
        for ii in range(n_snap): 
            if  snapshots[ii] is np.nan: 
                redshifts[ii] = np.nan
            else:
                redshifts[ii] = redshifts0[np.where(snapshots0 == snapshots[ii])][0]
    if return_opt is None: 
        return redshifts
    # elif return_opt == 'age':
    #     return age(redshifts)
    elif return_opt == 'FT':
        return FT(redshifts)

def _a_dot(a, h0, om_m, om_l):                                
    om_k = 1.0 - om_m - om_l      
    return h0 * a * np.sqrt(om_m * (a ** -3) + om_k * (a ** -2) + om_l)  

def _a_dot_recip(*args):
    return 1. / _a_dot(*args) 

# The formation time of stars in unit of Gyr
def FT(redshift, h0=0.6774, OmegaM=0.3089, OmegaL=0.6911):
    conv = units_conv.Unit("0.01 s Mpc km^-1").ratio('Gyr')
    redshift = 1./(1. + redshift)
    ns = np.size(redshift)
    if ns > 1:
        FT = np.zeros(ns)
        for ii in range(ns): FT[ii] = scipy.integrate.quad(_a_dot_recip, 0, redshift[ii], (h0, OmegaM, OmegaL))[0] * conv
    else:
        FT = scipy.integrate.quad(_a_dot_recip, 0, redshift, (h0, OmegaM, OmegaL))[0] * conv
    return FT

在Linux中，如果你想搜索一个文件夹中包含特定字段的文件，你可以使用 grep 命令。这个命令非常强大，可以在文件内容中搜索特定的文本模式。以下是一个基本的命令格式，用于在指定目录中搜索包含特定字段的文件：

grep -r "你的字段" /path/to/directory

grep 是搜索命令。
-r 表示递归搜索，这意味着 grep 将在指定目录及其所有子目录中查找。
"你的字段" 是你想搜索的文本。确保用引号包围，特别是如果文本中包含空格。
/path/to/directory 是你想搜索的目录的路径。

此命令将列出所有包含指定字段的文件及字段所在的行。如果你只想知道哪些文件包含该字段，而不关心具体的内容，可以使用 -l（小写L）选项：

grep -rl "你的字段" /path/to/directory

In [ ]:
'''在服务器上压缩文件'''
import zipfile
import os

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# 示例用法
'''要压缩文件所在的文件夹'''
folder_path = '/home/tnguser/pic/MRY_grp/nearby'
'''压缩文件存放的路径'''
zip_path = '/home/tnguser/pic/MRY_grp/nearby.zip'
zip_folder(folder_path, zip_path)

In [ ]:
'''显示hdf5文件的结构'''
import h5py

# Open the HDF5 file
file_meta = h5py.File('/home/tnguser/data/StarID/starID_1e7_gals_snap_40.hdf5', 'r')

# Function to recursively print the data structure
def print_structure(item, indent=""):
    if isinstance(item, h5py.Group):
        print(indent + "Group: " + item.name)
        for key in item.keys():
            print_structure(item[key], indent + "    ")
    elif isinstance(item, h5py.Dataset):
        print(indent + "Dataset: " + item.name)

# Call the function to print the structure
print_structure(file_meta)

# Close the file
file_meta.close()